In [1]:
!pip install datasets==3.6.0
!pip install transformers[sentencepiece] accelerate evaluate rouge_score conllu -q

from pathlib import Path
from datasets import load_dataset
from huggingface_hub import login, ModelCard, ModelCardData
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    pipeline,
)
import os, re, textwrap, evaluate
import numpy as np
import ast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.2 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00

In [2]:
dataset = load_dataset("universal_dependencies", "tr_boun", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

universal_dependencies.py: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7803 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/979 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/979 [00:00<?, ? examples/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 7803
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 979
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 979
    })
})


In [4]:
train_size = len(dataset['train'])
validation_size = len(dataset['validation'])
test_size = len(dataset['test'])

print(f"Eğitim veriseti boyutu: {train_size} cümle")
print(f"Doğrulama veriseti boyutu: {validation_size} cümle")
print(f"Test veriseti boyutu: {test_size} cümle")

Eğitim veriseti boyutu: 7803 cümle
Doğrulama veriseti boyutu: 979 cümle
Test veriseti boyutu: 979 cümle


In [5]:
pos_names = dataset['train'].features['upos'].feature.names

print("Kelime Türü Numaraları ve Karşılıkları:")
for i, name in enumerate(pos_names):
    print(f"Sayı: {i:<5} -> Metin: {name}")

Kelime Türü Numaraları ve Karşılıkları:
Sayı: 0     -> Metin: NOUN
Sayı: 1     -> Metin: PUNCT
Sayı: 2     -> Metin: ADP
Sayı: 3     -> Metin: NUM
Sayı: 4     -> Metin: SYM
Sayı: 5     -> Metin: SCONJ
Sayı: 6     -> Metin: ADJ
Sayı: 7     -> Metin: PART
Sayı: 8     -> Metin: DET
Sayı: 9     -> Metin: CCONJ
Sayı: 10    -> Metin: PROPN
Sayı: 11    -> Metin: PRON
Sayı: 12    -> Metin: X
Sayı: 13    -> Metin: _
Sayı: 14    -> Metin: ADV
Sayı: 15    -> Metin: INTJ
Sayı: 16    -> Metin: VERB
Sayı: 17    -> Metin: AUX


In [6]:
ilk_ornek = dataset['train'][333]
print("--- CÜMLENİN ORİJİNALİ ---")
print(ilk_ornek['text'])
print("\n--- CÜMLENİN PARÇALARI ---")

for i in range(len(ilk_ornek['tokens'])):
    kelime = ilk_ornek['tokens'][i]
    kok = ilk_ornek['lemmas'][i]
    tur_id = ilk_ornek['upos'][i]
    tur_metni = pos_names[tur_id]
    ekler = ilk_ornek['feats'][i]

    print(f"Kelime: {kelime:<15} | Kök: {kok:<15} | Tür: {tur_metni:<10} | Ek Bilgileri: {ekler}")

--- CÜMLENİN ORİJİNALİ ---
Ben titriyor, için için ağlıyor ve hiçbir şey yapamamanın ezikliğini yaşıyordum.

--- CÜMLENİN PARÇALARI ---
Kelime: Ben             | Kök: ben             | Tür: PRON       | Ek Bilgileri: {'Case': 'Nom', 'Number': 'Sing', 'Person': '1'}
Kelime: titriyor        | Kök: titre           | Tür: VERB       | Ek Bilgileri: {'Aspect': 'Prog', 'Number': 'Sing', 'Person': '3', 'Polarity': 'Pos', 'Tense': 'Pres'}
Kelime: ,               | Kök: ,               | Tür: PUNCT      | Ek Bilgileri: None
Kelime: için            | Kök: için            | Tür: ADV        | Ek Bilgileri: None
Kelime: için            | Kök: için            | Tür: ADP        | Ek Bilgileri: None
Kelime: ağlıyor         | Kök: ağla            | Tür: VERB       | Ek Bilgileri: {'Aspect': 'Prog', 'Number': 'Sing', 'Person': '3', 'Polarity': 'Pos', 'Tense': 'Pres'}
Kelime: ve              | Kök: ve              | Tür: CCONJ      | Ek Bilgileri: None
Kelime: hiçbir          | Kök: hiçbir          | Tür

In [7]:
model_name = "bigscience/mt0-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

In [8]:
UD_TO_TR_POS_MAP = {
    "NOUN": "isim", "VERB": "fiil", "ADJ": "sıfat", "ADV": "zarf",
    "PRON": "zamir", "PROPN": "özel isim", "NUM": "sayı", "ADP": "edat",
    "CCONJ": "bağlaç", "AUX": "yardımcı fiil", "DET": "belirteç",
    "PART": "partikül", "SCONJ": "alt-bağlaç", "INTJ": "ünlem",
}


UD_FEATURE_TO_TR_MAP = {
    "Case=Acc": "-i(belirtme)", "Case=Dat": "-e(yönelme)", "Case=Loc": "-de(bulunma)",
    "Case=Abl": "-den(ayrılma)", "Case=Gen": "-in(tamlayan)", "Case=Ins": "-le(vasıta)",
    "Case=Nom": "(yalın)",
    "Number=Sing": "(tekil)", "Number=Plur": "-ler(çoğul)",
    "Number[psor]=Sing": "(iyelik tekil)", "Number[psor]=Plur": "(iyelik çoğul)",
    "Person=1": "(1.kişi)", "Person=2": "(2.kişi)", "Person=3": "(3.kişi)",
    "Person[psor]=1": "(1.kişi iyelik)", "Person[psor]=2": "(2.kişi iyelik)", "Person[psor]=3": "(3.kişi iyelik)",
    "Tense=Pres": "-(i)yor(şimdiki z.)", "Tense=Past": "-di(geçmiş z.)", "Tense=Fut": "-ecek(gelecek z.)",
    "Tense=Pqp": "-mişti(öğ. geç. hik.)",
    "Aspect=Perf": "(belirli geçmiş)", "Aspect=Prog": "-mekte(sürmekte)",
    "Aspect=Hab": "-er(geniş z.)",
    "Polarity=Neg": "-me(olumsuzluk)", "Polarity=Pos": "(olumlu)",
    "VerbForm=Vnoun": "-mek(isim-fiil)", "VerbForm=Part": "-en(sıfat-fiil)", "VerbForm=Conv": "-erek(zarf-fiil)",
    "Mood=Imp": "(emir)", "Mood=Opt": "-e(istek)", "Mood=Cnd": "-se(şart)",
    "Mood=Pot": "-ebil(yeterlilik)", "Mood=Gen": "(genel kip)",
    "Voice=Pass": "-il(edilgen)", "Voice=Rcp": "-iş(işteş)", "Voice=Cau": "-dir(ettirgen)",
    "PronType=Prs": "(kişi zamiri)"
}

POS_NAMES = dataset['train'].features['upos'].feature.names

def prepare_data_for_training(dataset_batch):
    inputs, targets = [], []
    num_rows = len(dataset_batch[list(dataset_batch.keys())[0]])

    for i in range(num_rows):
        ornek = {key: dataset_batch[key][i] for key in dataset_batch}
        inputs.append(ornek['text'])

        analiz_parcalari = []
        for j in range(len(ornek['tokens'])):
            kelime = ornek['tokens'][j]
            tur_id = ornek['upos'][j]
            kok = ornek['lemmas'][j]
            ek_bilgileri_str = ornek['feats'][j]

            tur_kodu = POS_NAMES[tur_id]
            if tur_kodu in ['PUNCT', 'SYM', 'X']:
                continue

            if not kok or kok == '_':
                kok = kelime.lower()

            tur_adi = UD_TO_TR_POS_MAP.get(tur_kodu, tur_kodu.lower())
            root_pos_part = f"{kok}({tur_adi})"

            ek_parcalari = []
            if ek_bilgileri_str and ek_bilgileri_str != '_':
                feature_list = []
                if ek_bilgileri_str.strip().startswith('{'):
                    try:
                        feat_dict = ast.literal_eval(ek_bilgileri_str)
                        feature_list = [f"{key}={val}" for key, val in feat_dict.items()]
                    except (ValueError, SyntaxError):
                        feature_list = ek_bilgileri_str.split('|')
                else:
                    feature_list = ek_bilgileri_str.split('|')

                for ek in feature_list:
                    ek_temsili = UD_FEATURE_TO_TR_MAP.get(ek, f"({ek})")
                    ek_parcalari.append(ek_temsili)

            ek_parcalari.sort()

            tum_parcalar = [root_pos_part] + ek_parcalari
            final_kelime_analizi = " + ".join(tum_parcalar)

            satir = f"{kelime} -> {final_kelime_analizi}"
            analiz_parcalari.append(satir)

        targets.append(" | ".join(analiz_parcalari))

    return {"input": inputs, "target": targets}

In [9]:
processed_dataset = dataset.map(
    prepare_data_for_training,
    batched=True,
    remove_columns=dataset['train'].column_names
)


Map:   0%|          | 0/7803 [00:00<?, ? examples/s]

Map:   0%|          | 0/979 [00:00<?, ? examples/s]

Map:   0%|          | 0/979 [00:00<?, ? examples/s]

In [10]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input'], max_length=512, truncation=True)


    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target'], max_length=512, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


tokenized_dataset = processed_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=processed_dataset['train'].column_names
)

Map:   0%|          | 0/7803 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/979 [00:00<?, ? examples/s]

Map:   0%|          | 0/979 [00:00<?, ? examples/s]

In [11]:
rouge_metric = evaluate.load("rouge")

def normalize_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Label pad tokenlerini -100'den tokenizer.pad_token_id ile değiştir
    labels_with_padding = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Token ID'leri stringe çevir
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels_with_padding, skip_special_tokens=True)

    # Normalize edilmiş metinler
    decoded_preds = [normalize_text(p) for p in decoded_preds]
    decoded_labels = [normalize_text(l) for l in decoded_labels]

    # Rouge hesapla
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)
    rouge_result = {key: value * 100 for key, value in rouge_result.items()} # .mid.fmeasure kaldırıldı

    # Kök, POS, affix ve cümle doğrulukları
    total_words_evaluated = 0
    correct_root_count = 0
    correct_pos_count = 0
    correct_affix_count = 0
    correct_word_exact = 0
    correct_full_match_sentence = 0

    for pred_text, label_text in zip(decoded_preds, decoded_labels):
        pred_lines = [p.strip() for p in pred_text.split('|') if p.strip()]
        label_lines = [l.strip() for l in label_text.split('|') if l.strip()]

        # Cümledeki kelime sayılarını eşitlemek yerine, ne kadar varsa o kadarını değerlendir
        num_words_to_compare = min(len(pred_lines), len(label_lines))
        total_words_evaluated += len(label_lines) # Gerçek etiketteki kelime sayısını baz alalım

        for i in range(num_words_to_compare):
            pred_word = pred_lines[i]
            label_word = label_lines[i]

            # Kök-POS-affix ayır
            # Örnek: "git(fiil) + -ecek(gelecek z.) + (1.kişi)"
            pred_parts = [p.strip() for p in pred_word.split('->')[-1].split('+')]
            label_parts = [l.strip() for l in label_word.split('->')[-1].split('+')]

            if len(pred_parts) < 1 or len(label_parts) < 1:
                continue # Hatalı formatta üretilmiş kelime analizini atla

            pred_root_pos = pred_parts[0]
            label_root_pos = label_parts[0]

            # Kök ve POS doğruluğu
            if pred_root_pos == label_root_pos:
                correct_root_count += 1
                correct_pos_count += 1

            # Affix F1
            pred_affix_set = set(pred_parts[1:])
            label_affix_set = set(label_parts[1:])

            tp = len(pred_affix_set.intersection(label_affix_set))
            if not pred_affix_set and not label_affix_set: # İkisinde de ek yoksa, doğru kabul et
                 correct_affix_count += 1
            elif pred_affix_set or label_affix_set:
                precision = tp / len(pred_affix_set) if pred_affix_set else 0
                recall = tp / len(label_affix_set) if label_affix_set else 0
                if precision + recall > 0:
                    f1 = 2 * (precision * recall) / (precision + recall)
                    correct_affix_count += f1

            # Kelime exact match
            if pred_word == label_word:
                correct_word_exact += 1

        # Cümle exact match
        if pred_text == label_text:
            correct_full_match_sentence += 1

    # Sonuçları hesapla
    num_sentences = len(decoded_labels) if decoded_labels else 1
    total_words_evaluated = total_words_evaluated if total_words_evaluated > 0 else 1

    metrics = {
        "rouge1": round(rouge_result.get("rouge1", 0), 4),
        "rouge2": round(rouge_result.get("rouge2", 0), 4),
        "rougeL": round(rouge_result.get("rougeL", 0), 4),
        "root_pos_acc": round(correct_root_count / total_words_evaluated, 4),
        "affix_f1": round(correct_affix_count / total_words_evaluated, 4),
        "word_exact_match": round(correct_word_exact / total_words_evaluated, 4),
        "sentence_exact_match": round(correct_full_match_sentence / num_sentences, 4)
    }

    return metrics

In [12]:
hf_kullanici_adi = "obenadak"
hf_model_adi = "turkce-morfolojik-analiz-mt0-small"

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

batch_size = 8

training_args = Seq2SeqTrainingArguments(
    output_dir=hf_model_adi,
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=2,
    hub_model_id=f"{hf_kullanici_adi}/{hf_model_adi}",
    hub_strategy="end"
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

print(f"\n Model '{hf_model_adi}' klasörüne kaydedildi.")

/tmp/ipython-input-3391337965.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nebnebo (nebnebo-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Root Pos Acc,Affix F1,Word Exact Match,Sentence Exact Match
1,0.823500,0.258775,17.568600,13.559600,17.365300,0.041400,0.057700,0.016300,0.000000
2,0.323600,0.203823,17.661800,13.881100,17.472200,0.047800,0.062000,0.018900,0.000000
3,0.266100,0.176962,17.735100,14.187500,17.543000,0.055200,0.063600,0.022600,0.000000
4,0.238900,0.165271,17.841000,14.339900,17.659100,0.056900,0.063700,0.024100,0.000000
5,0.227100,0.160540,17.820700,14.299800,17.628900,0.056400,0.064300,0.024300,0.000000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].



 Model 'turkce-morfolojik-analiz-mt0-small' klasörüne kaydedildi.


In [13]:
test_sonuclari = trainer.evaluate(
    eval_dataset=tokenized_dataset["test"],
    metric_key_prefix="test"
)

print("\n--- TEST VERİ SETİ SONUÇLARI ---")
for key, value in test_sonuclari.items():
    print(f"{key}: {value:.4f}")


--- TEST VERİ SETİ SONUÇLARI ---
test_loss: 0.1581
test_rouge1: 17.9301
test_rouge2: 14.2752
test_rougeL: 17.7252
test_root_pos_acc: 0.0550
test_affix_f1: 0.0620
test_word_exact_match: 0.0232
test_sentence_exact_match: 0.0000
test_runtime: 53.7059
test_samples_per_second: 18.2290
test_steps_per_second: 2.2900
epoch: 5.0000


In [14]:
best_model_path = trainer.state.best_model_checkpoint
analiz_cihazi = pipeline("text2text-generation", model=best_model_path, tokenizer=tokenizer)


test_cumleleri = [
    "Çok güneşli bir günde yapılacak en iyi şey evde kalmaktır.",
    "Çay içmeye bayılıyorum.",
    "Gelecek hafta sonu için planların neler?",
    "Evden çıkarken ışıkları kapatmayı unutmuşum.",
]

def analiz_et_ve_yazdir(cumle):
    sonuc = analiz_cihazi(cumle, max_length=512)[0]['generated_text']

    print("-" * 50)
    print(f"Girdi: {cumle}")
    print("Modelin Analizi:")
    print(sonuc)
    print("-" * 50)

for cumle in test_cumleleri:
    analiz_et_ve_yazdir(cumle)

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------
Girdi: Çok güneşli bir günde yapılacak en iyi şey evde kalmaktır.
Modelin Analizi:
Çok -> Çok(zarf) + (None) | güneşli -> güneş(isim) + (3.kişi) + (tekil) + (yalın) | bir -> bir(belirteç) + (NumType=Card) | günde -> gün(isim) + (3.kişi) + (tekil) + -de(bulunma) | yapılacak -> yap(fiil) + (3.kişi) + (Evident=Fh) + (belirli geçmiş) + (olumlu) + (tekil) + -di(geçmiş z.) + -en(sıfat-fiil) | en -> en(zarf) + (None) | iyi -> iyi(sıfat) + (None) | şey -> şey(isim) + (3.kişi) + (tekil) + (yalın) | evde ->evde(zarf) + (3.kişi) + (tekil) + -de(bulunma) | kalmaktır -> kal(fiil) + (3.kişi) + (Evident=Fh) + (belirli geçmiş)
--------------------------------------------------


Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------
Girdi: Çay içmeye bayılıyorum.
Modelin Analizi:
Çay -> çay(isim) + (3.kişi) + (tekil) + (yalın) | içmeye -> iç(fiil) + (3.kişi) + (Evident=Fh) + (belirli geçmiş) + (olumlu) + (tekil) + -di(geçmiş z.) | bayılıyorum -> bayılıyor(fiil) + (3.kişi) + (Evident=Fh) + (belirli geçmiş) + (olumlu) + (tekil) + -di(geçmiş z.)
--------------------------------------------------


Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------
Girdi: Gelecek hafta sonu için planların neler?
Modelin Analizi:
Gelecek -> gel(fiil) + (3.kişi) + (Evident=Fh) + (belirli geçmiş) + (olumlu) + (tekil) + -di(geçmiş z.) | hafta -> hafta(isim) + (3.kişi) + (tekil) + (yalın) | sonu -> son(sıfat) + (3.kişi) + (tekil) + (yalın) | için -> için(edat) + (None) | planların -> plan(isim) + (3.kişi iyelik) + (3.kişi) + (iyelik tekil) + (tekil) + -in(tamlayan) | neler -> ne(zamir) + (3.kişi) + (tekil) + -(i)yor(şimdiki z.) + -ler(çoğul)
--------------------------------------------------
--------------------------------------------------
Girdi: Evden çıkarken ışıkları kapatmayı unutmuşum.
Modelin Analizi:
Evden -> ev(isim) + (3.kişi) + (tekil) + -den(ayrılma) | çıkarken -> çıkar(fiil) + (3.kişi) + (Evident=Fh) + (belirli geçmiş) + (olumlu) + (tekil) + -di(geçmiş z.) | ışıkları -> ışık(isim) + (3.kişi) + -i(belirtme) + -ler(çoğul) | kapatmayı -> kapat(fiil) + (3.kişi) + (Evident=Fh) + (belirli geçm

In [16]:
from huggingface_hub import upload_folder

login()

trainer.save_model(hf_model_adi)
print(f"Model ve tokenizer '{hf_model_adi}' klasörüne kaydedildi.")



hyperparameters_markdown = f"""
- **Base Model:** `{model_name}`
- **Epochs:** `{training_args.num_train_epochs}`
- **Training Batch Size:** `{training_args.per_device_train_batch_size}`
- **Evaluation Batch Size:** `{training_args.per_device_eval_batch_size}`
- **Optimizer:** AdamW (varsayılan)
- **Learning Rate:** `{training_args.learning_rate}`
- **Dataset:** `universal_dependencies` (tr_boun)
"""

validation_sonuclari = {}
for log in reversed(trainer.state.log_history):
    if 'eval_loss' in log:
        validation_sonuclari = {f"validation_{k.replace('eval_', '')}": v for k, v in log.items() if k not in ['step', 'epoch']}
        break

print("\n--- Final Validation Sonuçları ---")
for key, value in validation_sonuclari.items():
    print(f"{key}: {value:.4f}")

def create_results_table(results_dict):
    header_map = {
        'loss': 'Loss', 'rouge1': 'Rouge1', 'rouge2': 'Rouge2', 'rougeL': 'RougeL',
        'root_pos_acc': 'Root & POS Accuracy', 'affix_f1': 'Affix F1-Score',
        'word_exact_match': 'Word Exact Match', 'sentence_exact_match': 'Sentence Exact Match',
        'runtime': 'Runtime (s)',
        'samples_per_second': 'Samples / Second',
        'steps_per_second': 'Steps / Second'
    }
    markdown = "| Metrik | Puan |\n|---|---|\n"
    for key, value in results_dict.items():
        if '_' in key:
            clean_key = key.split('_', 1)[1]
        else:
            clean_key = key

        metric_name = header_map.get(clean_key, clean_key.replace("_", " ").title())

        if isinstance(value, (int, float)):
            markdown += f"| {metric_name} | {value:.4f} |\n"

    return markdown

validation_sonuclari_markdown = create_results_table(validation_sonuclari)
test_sonuclari_markdown = create_results_table(test_sonuclari)

# README.md oluştur
readme_path = Path(hf_model_adi) / "README.md"
readme_path.parent.mkdir(parents=True, exist_ok=True)

kart_icerigi = textwrap.dedent(f"""
---
license: apache-2.0
language:
- tr
datasets:
- universal_dependencies
tags:
- turkish
- morphological-analysis
- seq2seq
- mt0
---

# Türkçe Morfolojik Analiz Modeli: {hf_model_adi}

Bu model, Türkçe cümlelerin morfolojik analizini yapmak üzere `{model_name}` modelinin `universal_dependencies` (`tr_boun` alt kümesi) veri seti üzerinde ince ayarlanmasıyla (fine-tuning) eğitilmiştir.

Model, bir cümledeki her kelimeyi alıp kökünü, kelime türünü (Part-of-Speech) ve aldığı ekleri tahmin eder. Çıktı formatı aşağıdaki gibidir:

`Kelime -> kök(tür) + ek_1 + ek_2 ...`

## Eğitim Hiperparametreleri
{hyperparameters_markdown}

---

## Modelin Kullanımı

```python
from transformers import pipeline

analiz_cihazi = pipeline("text2text-generation", model="{hf_kullanici_adi}/{hf_model_adi}")
cumle = "Gelecek hafta sonu için planların neler?"
sonuc = analiz_cihazi(cumle, max_length=512)
print(sonuc[0]['generated_text'])

# Beklenen Çıktı (Örnek):
# Gelecek -> gel(fiil) + -ecek(gelecek z.) | hafta -> hafta(isim) + (yalın) | sonu -> son(isim) + (3.kişi iyelik) + (yalın) | için -> için(edat) | planların -> plan(isim) + -ler(çoğul) + (2.kişi iyelik) + (yalın) | neler -> ne(zamir) + -ler(çoğul)
# Değerlendirme Sonuçları (Evaluation Results)

Model, **tr_boun** veri setinin doğrulama (validation) ve test kümelerinde aşağıdaki sonuçları elde etmiştir.

## Final Doğrulama (Validation) Sonuçları
{validation_sonuclari_markdown}

## Test Seti Sonuçları
{test_sonuclari_markdown}

---
## Veri Hazırlığı (Data Preprocessing)

Girdi olarak cümlenin ham metni (**text**) kullanılır.
Hedef (**target**) ise, **universal_dependencies** veri setindeki **lemmas**, **upos** ve **feats** sütunları kullanılarak yapılandırılmış bir dizedir.
Noktalama işaretleri analizden çıkarılmıştır.
""")

readme_path.write_text(kart_icerigi, encoding="utf-8")

upload_folder(
    folder_path=hf_model_adi,
    repo_id=f"{hf_kullanici_adi}/{hf_model_adi}",
    commit_message="Epoch sayısı 5'e çıkarıldı, ROUGE use_stemmer=False olarak güncellendi."
)

Model ve tokenizer 'turkce-morfolojik-analiz-mt0-small' klasörüne kaydedildi.

--- Final Validation Sonuçları ---
validation_loss: 0.1605
validation_rouge1: 17.8207
validation_rouge2: 14.2998
validation_rougeL: 17.6289
validation_root_pos_acc: 0.0564
validation_affix_f1: 0.0643
validation_word_exact_match: 0.0243
validation_sentence_exact_match: 0.0000
validation_runtime: 54.1345
validation_samples_per_second: 18.0850
validation_steps_per_second: 2.2720


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...small/checkpoint-4880/rng_state.pth:  78%|#######7  | 11.0kB / 14.2kB            

  ...-small/checkpoint-4880/spiece.model:  95%|#########5| 4.11MB / 4.31MB            

...

  ...mall/checkpoint-4880/tokenizer.json:   3%|2         |  433kB / 16.3MB            

  ...small/checkpoint-3904/rng_state.pth:  78%|#######7  | 11.0kB / 14.2kB            

  ...l/checkpoint-4880/model.safetensors:   0%|          | 19.7kB / 1.20GB            

  ...-analiz-mt0-small/model.safetensors:   0%|          | 19.7kB / 1.20GB            

CommitInfo(commit_url='https://huggingface.co/obenadak/turkce-morfolojik-analiz-mt0-small/commit/4eac45ccc49913b2e7b7a4de425b287de203f9be', commit_message="Epoch sayısı 5'e çıkarıldı, ROUGE use_stemmer=False olarak güncellendi.", commit_description='', oid='4eac45ccc49913b2e7b7a4de425b287de203f9be', pr_url=None, repo_url=RepoUrl('https://huggingface.co/obenadak/turkce-morfolojik-analiz-mt0-small', endpoint='https://huggingface.co', repo_type='model', repo_id='obenadak/turkce-morfolojik-analiz-mt0-small'), pr_revision=None, pr_num=None)

In [ ]:
#trainer.push_to_hub(commit_mesaji = "Epoch sayısı 5'e çıkarıldı, ROUGE use_stemmer=False olarak güncellendi.")